In [11]:
import cv2
import numpy as  np
import tensorflow as tf
import tensorflow.keras
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,concatenate,AveragePooling2D,Rescaling,Conv2D,MaxPooling2D,BatchNormalization,LeakyReLU,GlobalAvgPool2D

In [12]:
def crop_minAreaRect(rect, box, gray):
    mult = 1 
    W = rect[1][0]
    H = rect[1][1]

    Xs = [i[0] for i in box]
    Ys = [i[1] for i in box]
    x1 = min(Xs)
    x2 = max(Xs)
    y1 = min(Ys)
    y2 = max(Ys)

    rotated = False
    angle = rect[2]

    if angle < -45:
        angle+=90
        rotated = True

    center = (int((x1+x2)/2), int((y1+y2)/2))
    size = (int(mult*(x2-x1)),int(mult*(y2-y1)))

    M = cv2.getRotationMatrix2D((size[0]/2, size[1]/2), angle, 1.0)

    cropped = cv2.getRectSubPix(gray, size, center)    
    cropped = cv2.warpAffine(cropped, M, size)

    croppedW = W if not rotated else H 
    croppedH = H if not rotated else W

    croppedRotated = cv2.getRectSubPix(cropped, (int(croppedW*mult), int(croppedH*mult)), (size[0]/2, size[1]/2))
    
    return croppedRotated

In [13]:
def conv_block(x, filters, kernel_size, strides, name=None):
  x = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', name=name, use_bias=False)(x)
  x = BatchNormalization(epsilon=0.001, name=name if name is None else '%s_BatchNorm' %name)(x)
  x = LeakyReLU(alpha=0.1, name=name if name is None else '%s_LeakyRelu' %name)(x)
  return x

def residual_block(x, filters_1, filters_2, name_1, name_2):
  short_cut = x
  x = conv_block(x, filters_1, (1,1), (1,1), name=name_1)
  x = conv_block(x, filters_2, (3,3), (1,1), name=name_2)
  residual_output = short_cut + x
  return residual_output

In [14]:
input_shape = (64, 64, 3)


inputs = tf.keras.Input(shape=input_shape)

x = conv_block(inputs, filters=8, kernel_size=(3,3), strides=(1,1), name='conv1_3x3')
x = conv_block(x, filters=16, kernel_size=(3,3), strides=(2,2), name='conv2_3x3')

x = residual_block(x, filters_1=8, filters_2=16, name_1='residual1_1x1', name_2='residual1_3x3')

x = conv_block(x, filters=32, kernel_size=(3,3), strides=(2,2), name='conv4_3x3')

for i in range(2):
  x = residual_block(x, filters_1=16, filters_2=32, name_1='residual2_'+ str(i) + '_1x1', name_2='resdual2_'+ str(i) + '_3x3')

x = conv_block(x, filters=64, kernel_size=(3,3), strides=(2,2), name='conv5_3x3')

for i in range(8):
  x = residual_block(x, filters_1=32, filters_2=64, name_1='residual3_'+ str(i) + '_1x1', name_2='resdual3_'+ str(i) + '_3x3')

x = conv_block(x, filters=128, kernel_size=(3,3), strides=(2,2), name='conv6_3x3')

for i in range(8):
  x = residual_block(x, filters_1=64, filters_2=128, name_1='residual4_'+ str(i) + '_1x1', name_2='resdual4_'+ str(i) + '_3x3')

x = conv_block(x, filters=256, kernel_size=(3,3), strides=(2,2), name='conv7_3x3')

for i in range(4):
  x = residual_block(x, filters_1=128, filters_2=256, name_1='residual5_'+ str(i) + '_1x1', name_2='resdual5_'+ str(i) + '_3x3')

x = GlobalAvgPool2D()(x)
x = Flatten()(x)
outputs = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_3x3 (Conv2D)             (None, 64, 64, 8)    216         ['input_3[0][0]']                
                                                                                                  
 conv1_3x3_BatchNorm (BatchNorm  (None, 64, 64, 8)   32          ['conv1_3x3[0][0]']              
 alization)                                                                                       
                                                                                                  
 conv1_3x3_LeakyRelu (LeakyReLU  (None, 64, 64, 8)   0           ['conv1_3x3_BatchNorm[0][0]

In [15]:
model = tf.keras.models.load_model("model_darknet53.h5")

In [16]:
from keras.preprocessing.image import img_to_array


In [17]:
img_crop=cv2.imread("2.jpg",1)
img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
img_crop = img_to_array(img_crop)
img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
Y_box_pred = model.predict(img_crop)
print(Y_box_pred)

[[0.99936163]]


In [18]:
video_path='test2.mp4'
cap=cv2.VideoCapture(video_path)
backSub=cv2.createBackgroundSubtractorMOG2()
if(cap.isOpened()==False):
    print("Khong mo duoc video")
while (cap.isOpened()):
    ret, frame=cap.read()
    if ret==True:
        #Thuat toan tru nen
        fgMask=backSub.apply(frame)
        fgMask=cv2.cvtColor(fgMask, 0)
        
        #Dung bo loc ma tran 5*5
        kernel = np.ones((7,7), np.uint8)
        #Lay gia tri pixel nho nhat cua kernel lam manh net
        fgMask=cv2.erode(fgMask,kernel,iterations=1)
        #Lay gia tri pixel lon nhat cua kernel lam lien net
        fgMask=cv2.dilate(fgMask,kernel,iterations=1)
        
        #tao them su lien net bang ham open
        fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel,iterations=1)
        #lay nguong 120
        _,fgMask = cv2.threshold(fgMask,120,255,cv2.THRESH_BINARY)
        #phat hien canh bang candy
        fgMask=cv2.Canny(fgMask, 20, 200)
        contours,_ = cv2.findContours(fgMask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(len(contours)):
            rect = cv2.minAreaRect(contours[i])
            if (rect[1][0]*rect[1][1]) < 1000:
                continue
            
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            img_crop = crop_minAreaRect(rect, box, frame)       
            img_crop = cv2.resize(img_crop, (64,64), interpolation = cv2.INTER_AREA)
            img_crop = img_to_array(img_crop)
            img_crop = img_crop.reshape((1, img_crop.shape[0], img_crop.shape[1], img_crop.shape[2]))
            cv2.drawContours(frame,[box],0,(0,255,0),2)
        
        
            Y_box_pred = model.predict(img_crop)
            if Y_box_pred <= 0.5:
                cv2.putText(frame, "Person", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            if Y_box_pred > 0.5:
                cv2.putText(frame, "Non", (box[2][0], box[2][1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        cv2.imshow('Frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cv2.destroyAllWindows()    
            